<a href="https://colab.research.google.com/github/ChiroDeniro/ai-ml-projecten/blob/main/RAG_Document_QA_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all required packages
!pip install -q transformers sentence-transformers chromadb langchain langchain-community pypdf

print("✅ All packages installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.3 MB/s et

RAG ENGINE

In [2]:
"""
RAG Engine - Free & Local
No API keys required!
"""

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

class FreeRAGEngine:
    def __init__(self):
        """Initialize RAG with free models"""
        print("🔧 Initializing RAG Engine...")

        # Embedding model
        print("📥 Loading embedding model (this takes ~1 minute first time)...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # QA model
        print("📥 Loading Q&A model...")
        self.qa_model = pipeline(
            "question-answering",
            model="deepset/roberta-base-squad2",
            device=0  # Use GPU!
        )

        self.vectorstore = None
        print("✅ RAG Engine ready!")

    def load_text(self, text, doc_name="document"):
        """Load text directly"""
        print(f"\n📄 Processing text...")

        # Split into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )

        chunks = text_splitter.split_text(text)
        print(f"✅ Created {len(chunks)} chunks")

        # Create vector store
        print("🗄️ Creating vector database...")
        self.vectorstore = Chroma.from_texts(
            texts=chunks,
            embedding=self.embeddings,
            metadatas=[{"source": doc_name, "chunk": i} for i in range(len(chunks))]
        )
        print("✅ Vector database ready!")

        return len(chunks)

    def ask(self, question, top_k=3):
        """Ask a question"""
        if not self.vectorstore:
            return {"answer": "Please load documents first!", "confidence": 0}

        # Retrieve relevant chunks
        print(f"\n🔍 Searching for: '{question}'")
        docs = self.vectorstore.similarity_search(question, k=top_k)

        if not docs:
            return {"answer": "No relevant information found.", "confidence": 0}

        # Combine context
        context = "\n\n".join([doc.page_content for doc in docs])

        # Get answer
        print("🤔 Generating answer...")
        try:
            result = self.qa_model(
                question=question,
                context=context[:2000]
            )

            print(f"✅ Answer found! (Confidence: {result['score']:.1%})")

            return {
                'answer': result['answer'],
                'confidence': result['score'],
                'context': context[:300] + "...",
                'sources': [f"Chunk {doc.metadata.get('chunk', '?')}" for doc in docs]
            }

        except Exception as e:
            return {
                'answer': f"Error: {str(e)}",
                'confidence': 0,
                'context': context[:300]
            }

print("🎉 RAG Engine class created!")

🎉 RAG Engine class created!


SAMPLE TEXT

In [3]:
# Sample document about AI
sample_document = """
Artificial Intelligence and Machine Learning

Machine learning is a subset of artificial intelligence that enables computers
to learn from data without being explicitly programmed. It has revolutionized
many industries including healthcare, finance, and transportation.

Deep learning is a specialized type of machine learning that uses neural networks
with multiple layers. These deep neural networks can automatically learn
hierarchical representations of data, making them exceptionally powerful for
complex tasks like image recognition and natural language processing.

Natural language processing (NLP) is a branch of AI focused on enabling computers
to understand, interpret, and generate human language. Modern NLP systems use
transformer architectures like BERT, GPT, and T5. These models have achieved
remarkable results in tasks such as translation, summarization, and question answering.

Computer vision allows machines to interpret and understand visual information
from the world. Applications include facial recognition, autonomous vehicles,
medical image analysis, and quality control in manufacturing. Recent advances
in convolutional neural networks have dramatically improved computer vision capabilities.

The transformer architecture, introduced in the paper "Attention is All You Need"
in 2017, has become the foundation for most modern AI systems. It uses
self-attention mechanisms to process sequential data more effectively than
previous recurrent neural network approaches.

Ethical considerations in AI include algorithmic bias, privacy concerns,
job displacement, and the need for transparency in automated decision-making.
Researchers, policymakers, and companies are working together to address these
important challenges as AI becomes increasingly prevalent in society.

Reinforcement learning is another important paradigm where agents learn to make
decisions by interacting with an environment and receiving rewards or penalties.
This approach has been successful in game playing (like AlphaGo), robotics,
and resource optimization problems.
"""

print("📄 Sample document created!")
print(f"Document length: {len(sample_document)} characters")

📄 Sample document created!
Document length: 2096 characters


INITIALIZE RAG & LOAD DOCUMENT

In [4]:
# Initialize RAG engine
rag = FreeRAGEngine()

# Load the sample document
num_chunks = rag.load_text(sample_document, "AI Overview")

print(f"\n🎉 RAG system ready with {num_chunks} chunks!")

🔧 Initializing RAG Engine...
📥 Loading embedding model (this takes ~1 minute first time)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


✅ RAG Engine ready!

📄 Processing text...
✅ Created 7 chunks
🗄️ Creating vector database...
✅ Vector database ready!

🎉 RAG system ready with 7 chunks!


ASK QUESTIONS

In [5]:
# Ask questions about the document
questions = [
    "What is machine learning?",
    "What is deep learning?",
    "What are some applications of computer vision?",
    "When was the transformer architecture introduced?",
    "What ethical concerns are mentioned about AI?"
]

print("=" * 70)
print("🎯 ASKING QUESTIONS ABOUT THE DOCUMENT")
print("=" * 70)

for q in questions:
    print("\n" + "─" * 70)
    result = rag.ask(q)

    print(f"\n❓ Question: {q}")
    print(f"💡 Answer: {result['answer']}")
    print(f"📊 Confidence: {result['confidence']:.1%}")
    print(f"📚 Sources: {', '.join(result['sources'])}")

🎯 ASKING QUESTIONS ABOUT THE DOCUMENT

──────────────────────────────────────────────────────────────────────

🔍 Searching for: 'What is machine learning?'
🤔 Generating answer...
✅ Answer found! (Confidence: 9.9%)

❓ Question: What is machine learning?
💡 Answer: enables computers 
to learn from data without being explicitly programmed
📊 Confidence: 9.9%
📚 Sources: Chunk 0, Chunk 1, Chunk 3

──────────────────────────────────────────────────────────────────────

🔍 Searching for: 'What is deep learning?'
🤔 Generating answer...
✅ Answer found! (Confidence: 25.0%)

❓ Question: What is deep learning?
💡 Answer: a specialized type of machine learning
📊 Confidence: 25.0%
📚 Sources: Chunk 1, Chunk 0, Chunk 3

──────────────────────────────────────────────────────────────────────

🔍 Searching for: 'What are some applications of computer vision?'
🤔 Generating answer...
✅ Answer found! (Confidence: 5.1%)

❓ Question: What are some applications of computer vision?
💡 Answer: facial recognition, auto

TRY QUESTIONS

In [6]:
# Ask your own question!
my_question = "What is NLP?"  # Change this to whatever you want!

result = rag.ask(my_question)

print("=" * 70)
print(f"❓ Your Question: {my_question}")
print("=" * 70)
print(f"\n💡 Answer: {result['answer']}")
print(f"\n📊 Confidence: {result['confidence']:.1%}")
print(f"\n📝 Context used:")
print(result['context'])


🔍 Searching for: 'What is NLP?'
🤔 Generating answer...
✅ Answer found! (Confidence: 35.7%)
❓ Your Question: What is NLP?

💡 Answer: Natural language processing

📊 Confidence: 35.7%

📝 Context used:
Natural language processing (NLP) is a branch of AI focused on enabling computers 
to understand, interpret, and generate human language. Modern NLP systems use 
transformer architectures like BERT, GPT, and T5. These models have achieved 
remarkable results in tasks such as translation, summarizati...


ADD DOCUMENTS

In [7]:
# Add your own text here!
my_document = """
Paste your own text here! It can be:
- Notes from your econometrics course
- A Wikipedia article
- Research paper abstract
- Anything you want to ask questions about!
"""

# Load it
rag.load_text(my_document, "My Document")

# Ask questions about YOUR content
my_result = rag.ask("What is this document about?")
print(f"Answer: {my_result['answer']}")


📄 Processing text...
✅ Created 1 chunks
🗄️ Creating vector database...
✅ Vector database ready!

🔍 Searching for: 'What is this document about?'
🤔 Generating answer...
✅ Answer found! (Confidence: 1.7%)
Answer: Natural language processing
